# Introdución a APIs



In [ ]:
!pip install tweepy
import tweepy
import pandas as pd
from textblob import TextBlob

def pull_tweets (query, maxTweets = 1000, tweetsPerQuery = 100, max_id = -1, sinceId = None):
    '''
    Finds tweets (Comment, Date, Favorites, User) for a query string.
    Twitter API limit per query is 100. Combines these queries. 
    '''
    
    # Fill with your own app details
    API_KEY = "wCl2jflXpyWmYDM22iKFsaiS2"
    API_SEC = "f3DkCao13uCfA58bSQXahsDVNF5qzNztrgt3wB2RDDAV8zyXvT"
    
    # connect to Twitter using authentication
    auth = tweepy.AppAuthHandler(API_KEY, API_SEC)
    # wait_on_rate_limit means that if the API limit is hit, 
    #   the pulls will wait until more calls are available
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    
    # Pull comments from Twitter
    # See https://developer.twitter.com/en/docs/tweets/timelines/guides/working-with-timelines
    tweetCount = 0
    data = pd.DataFrame() 
    
    while tweetCount < maxTweets:
        if (max_id <= 0):
            new_tweets = api.search(q=query, count=tweetsPerQuery, 
                                    since_id=sinceId)
        else:
            new_tweets = api.search(q=query, count=tweetsPerQuery,
                                    max_id=str(max_id - 1), 
                                    since_id=sinceId)
        if not new_tweets:
            print("No more tweets found")
            break
    
        tweetCount += len(new_tweets)
        print("Downloaded {0} tweets".format(tweetCount))
        max_id = new_tweets[-1].id
                    
        ## Create a dataset from the downloaded tweets
        new_data = pd.DataFrame([{
                'Date': tweet.created_at,
                'Comments': tweet.text, 
                'User': tweet.user.name, 
                'Favorites': tweet.favorite_count} 
                for tweet in new_tweets])
          
        data = data.append(new_data)
    return data

In [ ]:
data = pull_tweets("trump", maxTweets = 5000)

# In real life you might have test data with pre-labeled sentiments. We will use a simple word net algorithm to classify for now.
data['Polarity'] = [TextBlob(comment).polarity for comment in data.Comments]

data.loc[data['Polarity'] > 0, 'Sentiment'] = 'positive'
data.loc[data['Polarity'] < 0, 'Sentiment'] = 'negative'
data.loc[data['Polarity'] == 0, 'Sentiment'] = 'neutral'

#convert data to a csv
data.to_csv("juegosPanamericanos.csv", index = False)

In [ ]:
data.head(50)